In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Dirichlet
from torch.distributions import Independent

In [13]:
random_x = torch.randn((30,20))
#random_x = 0.1*torch.ones((30,20))

In [14]:
head = Dirichlet(random_x)

In [15]:
out = head.rsample()

In [16]:
out.shape

torch.Size([30, 20])

In [17]:
out.sum(dim=-1)

tensor([1.0000e+00, 1.0000e+00, 2.3510e-37, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 2.3510e-37, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])

In [18]:
head.log_prob(out)

tensor([1429.9387,  860.2711, 1284.1647, 1110.5370,  960.1478, 1279.7449,
        1444.6833, 1763.0642, 1892.3984, 1922.1530, 1194.2577,  354.3149,
        1460.5626, 1094.7314, 1358.4684, 1551.8958, 1885.7937, 1514.9250,
         799.5903,  918.2299,  814.7458, 1430.4659,  952.2896, 1496.9838,
         811.9423, 1313.9857, 1676.3671, 1015.9489,  915.2290,  861.9023])

In [7]:
all(list((torch.abs(out.sum(dim=-1)-1) < 1.e-4).numpy()))

True

## エラーが出るようなパラメータ―を求める 

以下の例はgoogle colabではエラーがでるが，python3.7環境ではエラーが出ない．

In [8]:
counter_lim = 1000
counter = 0
while True:
    try:
        random_x = torch.randn((30,20))
        head = Dirichlet(random_x)
        out = head.rsample()
    except:
        print(random_x)
        
    counter += 1
    if counter > counter_lim:
        break

## サンプリング値がおかしい場合

In [9]:
counter_lim = 1000
counter = 0
while True:
    random_x = torch.randn((1,20))
    head = Dirichlet(random_x)
    out = head.rsample()
    if not all(list((torch.abs(out.sum(dim=-1)-1) < 1.e-4).numpy())):
        print(random_x)
        
    counter += 1
    if counter > counter_lim:
        break

tensor([[-1.0149e+00,  1.2589e+00,  4.2021e-01, -1.4810e+00, -1.3768e+00,
         -2.2268e-01,  5.1317e-01,  3.1320e-01,  1.6949e+00, -1.7917e+00,
          1.1850e+00,  1.7434e+00,  1.1275e+00,  2.1978e-01, -5.5611e-04,
          1.5695e+00,  9.3637e-01,  1.9708e-02, -1.1750e+00,  4.0988e-01]])
tensor([[-4.0661e-01,  1.7061e-01,  5.2716e-01, -1.6545e+00,  3.4036e-01,
         -8.3972e-01,  5.6470e-02,  3.7369e-01,  1.2939e+00, -6.1234e-01,
          2.3653e+00, -1.4958e-03,  1.3552e+00, -1.3564e+00, -1.4112e+00,
         -1.0127e-01, -1.5074e-01,  6.2986e-01, -9.9232e-01,  1.3461e+00]])
tensor([[-5.0793e-02,  1.6219e+00,  2.3441e-01,  2.8729e+00,  3.0034e-01,
         -1.0032e+00, -1.3569e+00, -7.6564e-01,  9.1640e-01,  7.0209e-01,
          1.9979e-01,  7.7838e-01,  1.2783e-01, -2.2607e-02, -5.1300e-02,
         -2.0921e+00,  1.6087e-01,  1.1385e-01,  1.3268e-01, -1.1928e-03]])
tensor([[-9.5895e-01, -1.7975e+00,  1.8372e+00, -1.5960e+00,  1.7533e+00,
         -1.6370e-01,  3.4621e-0

dirichlet分布のalphaの値は非負でなければならないのでabsをすればよい

In [10]:
counter_lim = 1000
counter = 0
while True:
    random_x = torch.abs(torch.randn((1,20)))
    head = Dirichlet(random_x)
    out = head.rsample()
    if not all(list((torch.abs(out.sum(dim=-1)-1) < 1.e-4).numpy())):
        print(random_x)
        
    counter += 1
    if counter > counter_lim:
        break

0のときは大丈夫

In [11]:
random_x = torch.zeros((30,20))
head = Dirichlet(random_x)
out = head.rsample()
out.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000])

In [12]:
head.log_prob(out)

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan])

しかし尤度の値がnanになる

## clipする場合 

In [24]:
random_x = 1.e-5*torch.ones((30,20))
head = Dirichlet(random_x)
out = head.rsample()
print(out.sum(dim=-1))
print(head.log_prob(out))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000])
tensor([-161.8273, -161.8273, -161.8273, -161.8273, -161.8273, -161.8273,
        -161.8273, 1437.6367, -161.8273, -161.8273, -161.8273, -161.8273,
        -161.8273, -161.8273, -161.8273, -161.8273, -161.8273, -161.8273,
        -161.8273, -161.8273, -161.8273, -161.8273, -161.8273, -161.8273,
        -161.8273, -161.8273, -161.8273, -161.8273, 1437.6367, -161.8273])


In [20]:
random_x = 1.e5*torch.ones((30,20))
head = Dirichlet(random_x)
out = head.rsample()
print(out.sum(dim=-1))
print(head.log_prob(out))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000])
tensor([140., 140., 136., 140., 138., 138., 136., 142., 136., 128., 140., 142.,
        136., 136., 136., 138., 142., 134., 140., 130., 142., 138., 136., 136.,
        138., 138., 132., 140., 140., 144.])


## モデルを利用する場合

In [7]:
class DirichletHead(nn.Module):
    def __init__(self):
        super(DirichletHead, self).__init__()
        
    def forward(self, alpha):
        return torch.distributions.Dirichlet(alpha)

In [8]:
class Policy(nn.Module):
    def __init__(self, in_channels=3, out_number=20):
        super(Policy, self).__init__()
        self.out_number = out_number
        
        self.conv1 = nn.Conv2d(in_channels, 12, kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, self.out_number, kernel_size=5, padding=2, stride=(2,3))
        self.bn2 = nn.BatchNorm2d(self.out_number)
        self.conv3 = nn.Conv2d(self.out_number, self.out_number, kernel_size=(4,3), padding=(2,1), stride=(2,3))
        self.bn3 = nn.BatchNorm2d(self.out_number)

        self.avgpool = nn.AvgPool2d(kernel_size=3)
        self.head = DirichletHead()
        
        
    def forward(self, x):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = self.avgpool(x)
        x = torch.reshape(x, (-1, self.out_number))
        #x = F.softmax(x, dim=-1)
        x = F.relu(x)
        x = torch.clamp(x, 0.01, 10)
        out = self.head(x)
        return out

In [9]:
random_x = torch.randn((30, 3, 20, 50))
policy = Policy(3, 20)

In [10]:
out = policy(random_x)
print(out.batch_shape, out.event_shape)
print(out.rsample(torch.Size([])).shape)

torch.Size([30]) torch.Size([20])
torch.Size([30, 20])
